# CH3133 - Computational Practicum - Q1 - Lecture coding 5

Import packages.

In [1]:
import time
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt
from typing import Callable

## Analytical solution - linear system of ODEs

Solve the following reaction system analytically.

$A \xrightleftharpoons[k_2]{k_1} B \xrightleftharpoons[k_4]{k_3} C$

Initial conditions:

$C_A(0)=1\, mol/L$

$C_B(0)=C_C(0)=0\, mol/L$

Rate constants:

$k_1=1\, min^{-1}, k_2=0\, min^{-1}, k_3=2\, min^{-1}, k_4=3\, min^{-1}$


Define parameters.

In [ ]:
# time domain
t = np.linspace(0, 10, 500)
dt = t[1]-t[0]

# initial condition
c_0 = np.array([1,0,0])

# matrix of the rates
K = np.array([[-1,0,0],[1,-2,3],[0,2,-3]])

Calculate exponential of matrix.

Note: `expm` vs `exp` in scipy
- `exmp`: matrix exponentiation $e^A=Ue^\lambda U^{-1}$
- `exp`: scalar value exponentiation $e^x, x \in \mathbb{R}$


In [ ]:
eK = K * dt
eKdt = sp.linalg.expm(eK)

Calculate concentration at time steps.

In [ ]:
C = [c_0]
for i in range(len(t)-1):
    c_0 = np.dot(eKdt,c_0)
    C.append(c_0)

# extract values
cA = [array[0] for array in C]
cB = [array[1] for array in C]
cC = [array[2] for array in C]

Plot the results.

In [ ]:
fig, ax = plt.subplots()
ax.plot(t, cA, label = "cA")
ax.plot(t, cB, label = "cB")
ax.plot(t, cC, label = "cC")
ax.set_xlabel("time /s")
ax.set_ylabel("concentration /mol/L")
ax.set_title("Analytical solution - linear system of ODEs")
ax.legend()
ax.grid()
fig.show()

## The Forward Euler method

We analyze the semi-batch reactor. The reactor model contains a non-autonomous term (right hand-side contains the independent variable $t$ explicitly).

$\frac{dC_A}{dt}=\frac{\dot{V}}{V_0+\dot{V}t}\left(C_A^{in}-C_A\right)-kC_A$

$\frac{dC_B}{dt}=kC_A$

Initial contitions:

$C_A(t=0\,s)=1\, mol/L,\; C_B(0)=0\, mol/L$


Define parameters.

In [ ]:
# time domain
t_start = 0.0
t_end = 30.0
n = 500
t = np.linspace(t_start, t_end, n)

# initial conditions
c0 = np.array([1,0])

Define the reaction system.

In [ ]:
def dC(t: np.ndarray, c: np.ndarray) -> np.ndarray:
    """ODE system for semi-batch reactor.

    Parameters
    ----------
    t : np.ndarray
        Time variable
    c : np.ndarray
        Concentration of individual components

    Returns
    -------
    np.ndarray
        Concentration gradient in reactor
    """
    cA, _ = c
    k = 0.2
    Vdot = 0.1
    V_0 = 10
    cA_in = 0.5
    dcA = Vdot/(V_0+Vdot*t)*(cA_in-cA)-k*cA
    dcB = k*cA
    return np.array([dcA, dcB])

Define forward euler method.

In [ ]:
def forward_euler(func: Callable, y0: np.ndarray, t: np.ndarray) -> np.ndarray:
    """Generic forward euler method for initial value problem.

    Parameters
    ----------
    func : Callable
        ODE system to be solved
    y0 : np.ndarray
        Initial condition
    t : np.ndarray
        Time grid points

    Returns
    -------
    np.ndarray
        Solution of ODE system
    """
    y = np.zeros((len(y0),len(t)))
    h = t[1]-t[0]
    # initial condition
    y[:,0] = y0
    for i in range(1,len(t)):
        y[:,i] = y[:,i-1]+h*func(t[i-1], y[:,i-1])
    return y

Execute forward euler method.

In [ ]:
c = forward_euler(dC, c0, t)

Plot the results.

In [ ]:
fig, ax = plt.subplots()
ax.plot(t, c[0,:], label = "cA")
ax.plot(t, c[1,:], label = "cB")
ax.set_xlabel("time /s")
ax.set_ylabel("concentration /mol/L")
ax.set_title("The Forward Euler method")
ax.legend()
ax.grid()
fig.show()

## The Backward Euler method

We analyze the semi-batch reactor. The reactor model contains a non-autonomous term (right hand-side contains the independent variable $t$ explicitly).

$\frac{dC_A}{dt}=\frac{\dot{V}}{V_0+\dot{V}t}\left(C_A^{in}-C_A\right)-kC_A$

$\frac{dC_B}{dt}=kC_A$

Initial contitions:

$C_A(t=0\,s)=1\, mol/L,\; C_B(0)=0\, mol/L$


Define parameters.

In [ ]:
# time domain
t_start = 0.0
t_end = 30.0
n = 500
t = np.linspace(t_start, t_end, n)

# initial conditions
c0 = np.array([1,0])

Define chemical reaction system.

In [ ]:
def dC(t: np.ndarray, c: np.ndarray) -> np.ndarray:
    """ODE system for semi-batch reactor.

    Parameters
    ----------
    t : np.ndarray
        Time variable
    c : np.ndarray
        Concentration of individual components

    Returns
    -------
    np.ndarray
        Concentration gradient in reactor
    """
    cA, _ = c
    k = 0.2
    Vdot = 0.1
    V_0 = 10
    cA_in = 0.5
    dcA = Vdot/(V_0+Vdot*t)*(cA_in-cA)-k*cA
    dcB = k*cA
    return np.array([dcA, dcB])

Define backward euler method.

In [ ]:
def backward_euler_fixed_point(func: Callable, y0: np.ndarray, t: np.ndarray,
                               tol:float=1e-6, max_iter:int=100) -> np.ndarray:
    """Solves the ODE y' = f(t, y) using the Backward (Implicit) Euler method
    with fixed-point iteration.

    Parameters
    ----------
    func : Callable
        Function that defines the ODE (y' = func(t, y)).
    y0 : np.ndarray
        Initial condition.
    t : np.ndarray
        Time domain.
    tol : float, optional
        Tolerance for the fixed-point iteration (default is 1e-6).
    max_iter : int, optional
        Maximum number of iterations for the fixed-point iteration (default is 100).

    Returns
    -------
    np.ndarray
        Array of solution values at the time points.
    """
    # Initialize arrays for time and solution values
    y = np.zeros([len(y0),len(t)])
    y[:,0] = y0
    dt = t[1]-t[0]

    # Iterate over each time step
    for n in range(len(t)-1):
        # Initial guess for y_{i+1}
        y_next = y[:,n]
        
        # Fixed-point iteration to solve for y_{i+1}
        for i in range(max_iter):
            y_new = y[:,n] + dt * func(t[n+1], y_next)
            if max(np.abs(y_new - y_next)) < tol:
                break
            y_next = y_new
        
        # Update solution
        y[:,n+1] = y_next
    
    return y

Execute backward euler method.

In [ ]:
c = backward_euler_fixed_point(dC, c0, t)

Plot the results.

In [ ]:
fig, ax = plt.subplots()
ax.plot(t, c[0,:], label = "cA")
ax.plot(t, c[1,:], label = "cB")
ax.set_xlabel("time /s")
ax.set_ylabel("concentration /mol/L")
ax.set_title("The Backward Euler method")
ax.legend()
ax.grid()
fig.show()

## Numerical error

Define parameters

In [ ]:
# ODE
def dcA(t, c):
    return -0.2*c

# initial condition
c0 = np.array([1])

# analytical solution of ODE
def cA_analytical(k: float, t: float) -> float:
    return np.exp(-k*t)

Define a fine and a coarse time grid.

In [ ]:
t_coarse = np.linspace(0,30,11)
t_fine = np.linspace(0,30,101)

Calculate ODE solution and measure execution time.

In [ ]:
start_coarse = time.time()
c_coarse = forward_euler(dcA, c0, t_coarse)
end_coarse = time.time()
time_coarse = end_coarse - start_coarse
print(f"Euler method for coarse grid took {round(time_coarse, 4)} seconds.")

start_fine = time.time()
c_fine = forward_euler(dcA, c0, t_fine)
end_fine = time.time()
time_fine = end_fine - start_fine
print(f"Euler method for fine grid took {round(time_fine, 4)} seconds.")

Plot the results.

In [ ]:
fig, ax = plt.subplots()
ax.plot(t_coarse, c_coarse[0,:], label = f"cA - h=3s - solution time={round(time_coarse, 4)}")
ax.plot(t_fine, c_fine[0,:], label = f"cA - h=0.3s - solution time={round(time_fine, 4)}")
ax.plot(t_fine, cA_analytical(0.2, t_fine), label = "cA - analytical solution")
ax.set_xlabel("time /s")
ax.set_ylabel("concentration /mol/L")
ax.set_title("Forward euler - grid comparison")
ax.legend()
ax.grid()
fig.show()

## Stability

Define parameters

In [ ]:
# ODE
def dcA(t, c):
    return -0.2*c

# initial condition
c0 = np.array([1])

# analytical solution of ODE
def cA_analytical(k: float, t: float) -> float:
    return np.exp(-k*t)

Define a fine and a coarse time grid.

In [ ]:
t_fine = np.linspace(0,50,51)
c_fine = forward_euler(dcA, c0, t_fine)
t_coarse = np.linspace(0,50,5)
c_coarse = forward_euler(dcA, c0, t_coarse)

Plot the results.

In [ ]:
fig, ax = plt.subplots()
ax.plot(t_fine, c_fine[0,:], label = f"cA - h=1s")
ax.plot(t_coarse, c_coarse[0,:], label = f"cA - h=12.5s")
ax.plot(t_fine, cA_analytical(0.2, t_fine), label = "cA - analytical solution")
ax.set_xlabel("time /s")
ax.set_ylabel("concentration /mol/L")
ax.set_title("Stability")
ax.legend()
ax.grid()
fig.show()

## SciPy's `solve_ivp`

We analyze the semi-batch reactor. The reactor model contains a non-autonomous term (right hand-side contains the independent variable $t$ explicitly).

$\frac{dC_A}{dt}=\frac{\dot{V}}{V_0+\dot{V}t}\left(C_A^{in}-C_A\right)-kC_A$

$\frac{dC_B}{dt}=kC_A$

Initial contitions:

$C_A(t=0\,s)=1\, mol/L,\; C_B(0)=0\, mol/L$


Define parameters.

In [ ]:
# time domain
t_start = 0.0
t_end = 30.0
n = 500
t = np.linspace(t_start, t_end, n)

# initial conditions
c0 = np.array([1,0])

Define the reaction system.

In [ ]:
def dC(t: np.ndarray, c: np.ndarray) -> np.ndarray:
    """ODE system for semi-batch reactor.

    Parameters
    ----------
    t : np.ndarray
        Time variable
    c : np.ndarray
        Concentration of individual components

    Returns
    -------
    np.ndarray
        Concentration gradient in reactor
    """
    cA, _ = c
    k = 0.2
    Vdot = 0.1
    V_0 = 10
    cA_in = 0.5
    dcA = Vdot/(V_0+Vdot*t)*(cA_in-cA)-k*cA
    dcB = k*cA
    return np.array([dcA, dcB])

Solve the IVP using SciPy's `solve_ivp` function.

In [ ]:
results = sp.integrate.solve_ivp(dC, (t[0],t[-1]), c0, t_eval=t)
cA = results.y[0]
cB = results.y[1]

Plot the results.

In [ ]:
fig, ax = plt.subplots()
ax.plot(t, cA, label = "cA")
ax.plot(t, cB, label = "cB")
ax.set_xlabel("time /s")
ax.set_ylabel("concentration /mol/L")
ax.set_title("IVP solution using solve_ivp")
ax.legend()
ax.grid()
fig.show()